# Scene Categorization

In [ ]:
import numpy as np
import pandas as pd
import cv2
import re
import time
from imageio import imread
from pathlib import Path
from scipy.io import loadmat
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn.neighbors import KDTree, KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from keras.applications import *

In [ ]:
def numerical_sort(value):
    numbers = re.compile(r'(\d+)')
    value = str(value)
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def get_images():
    train_path = Path('./data/train')
    test_path = Path('./data/test')
    X_train = []
    X_test = []
    for image_path in sorted(train_path.glob('*.jpg'),key=numerical_sort):
        image = imread(image_path)
        X_train.append(image)
    for image_path in sorted(test_path.glob('*.jpg'),key=numerical_sort):
        image = imread(image_path)
        X_test.append(image)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    return X_train,X_test

def get_labels():
    label_path = Path('./data/gs.mat')
    labels = loadmat(label_path)
    y_train = labels['train_gs'].flatten()
    y_test = labels['test_gs'].flatten()
    return y_train,y_test

def histogram_distance(h1,h2):
    h1 = h1.astype(np.float32)
    h2 = h2.astype(np.float32)
    method = cv2.HISTCMP_INTERSECT
    return cv2.compareHist(h1,h2,method)

def get_scores(model,X_train,y_train,X_test,y_test):
    yhat = model.predict(X_train)
    score = accuracy_score(y_train,yhat)
    print('Training Accuracy: %f' % round(score,3))    
    yhat = model.predict(X_test)
    score = accuracy_score(y_test,yhat)
    print('Test Accuracy: %f' % round(score,3))
    print('Confusion Matrix:\n', confusion_matrix(y_test,yhat))

In [ ]:
X_train,X_test = get_images()
print('training data size:',X_train.shape)
print('test data size:',X_test.shape)

y_train,y_test = get_labels()
print('training label size:',y_train.shape)
print('test label size:',y_test.shape)

## Color histogram and kNN classifier

In [ ]:
def get_color_hists(X_train,X_test,bins=10):
    X_train_hist = np.zeros((len(X_train),bins*3))
    X_test_hist = np.zeros((len(X_test),bins*3))
    for i in range(len(X_train)):
        red = cv2.calcHist([X_train[i,:,:,0]],[0],None,[bins],[0,256]).flatten()
        green = cv2.calcHist([X_train[i,:,:,1]],[0],None,[bins],[0,256]).flatten()
        blue = cv2.calcHist([X_train[i,:,:,2]],[0],None,[bins],[0,256]).flatten()
        X_train_hist[i] = np.concatenate([red,green,blue])
    for i in range(len(X_test)):
        red = cv2.calcHist([X_test[i,:,:,0]],[0],None,[bins],[0,256]).flatten()
        green = cv2.calcHist([X_test[i,:,:,1]],[0],None,[bins],[0,256]).flatten()
        blue = cv2.calcHist([X_test[i,:,:,2]],[0],None,[bins],[0,256]).flatten()
        X_test_hist[i] = np.concatenate([red,green,blue])
    return X_train_hist,X_test_hist

In [ ]:
X_train_hist,X_test_hist = get_color_hists(X_train,X_test,bins=10)

In [ ]:
model = KNeighborsClassifier(n_neighbors=10,n_jobs=-1)
model.fit(X_train_hist,y_train)
get_scores(model,X_train_hist,y_train,X_test_hist,y_test)

## Bag of visual words model and nearest neighbor classifier

In [ ]:
def get_sift():
    sift_path = Path('./data/sift_desc.mat')
    sift = loadmat(sift_path)
    train = sift['train_D'].flatten()
    test = sift['test_D'].flatten()
    train_sifts = [t.T for t in train]
    test_sifts = [t.T for t in test]
    return train_sifts,test_sifts

In [ ]:
train_sifts,test_sifts = get_sift()

In [ ]:
def vis_bow(train_sifts,test_sifts,n_clusters=100):
    stacked_train_sifts = np.vstack(train_sifts)
    stacked_test_sifts = np.vstack(test_sifts)
    kmeans = MiniBatchKMeans(n_clusters=n_clusters)
    kmeans.fit(stacked_train_sifts)
    train_clusters = [kmeans.predict(words) for words in train_sifts]
    X_train_bow = np.array([np.bincount(words, minlength=n_clusters) for words in train_clusters])
    test_clusters = [kmeans.predict(words) for words in test_sifts]
    X_test_bow = np.array([np.bincount(words, minlength=n_clusters) for words in test_clusters])
    return X_train_bow,X_test_bow

In [ ]:
X_train_bow,X_test_bow = vis_bow(train_sifts,test_sifts,n_clusters=100)

In [ ]:
model = KNeighborsClassifier(n_neighbors=15,n_jobs=-1)
model.fit(X_train_bow,y_train)
get_scores(model,X_train_bow,y_train,X_test_bow,y_test)

## Bag of visual words model and a discriminative classifier

In [ ]:
start = time.clock()
model = SVC(C=.001,kernel='linear')
model.fit(X_train_bow,y_train)
end = time.clock()
print('training time:',round(end-start,3),'seconds')

start = time.clock()
get_scores(model,X_train_bow,y_train,X_test_bow,y_test)
end = time.clock()
print('testing time:',round(end-start,3),'seconds')

## CNN model and a discriminative classifier

In [ ]:
#model = vgg16.VGG16(weights='imagenet', include_top=False)
X_train_encoding = model.predict(vgg16.preprocess_input(X_train),verbose=1,batch_size=32)
#X_test_encoding = model.predict(vgg16.preprocess_input(X_test),verbose=1,batch_size=1)

In [ ]:
X_train_encoding.shape,X_test_encoding.shape

## Graduate Points

In [ ]:
?model.predict